# RAG

In [1]:
from helper import *
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid

client = QdrantClient(path="embeddings")
downloaded_embedding_model_path = hf_hub_download(repo_id="mixedbread-ai/mxbai-embed-large-v1", filename="gguf/mxbai-embed-large-v1-f16.gguf")
llm_embedding = Llama(model_path=downloaded_embedding_model_path, n_gpu_layers=-1, embedding=True, verbose=False)

downloaded_model_path = hf_hub_download(repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF", filename="Llama-3.2-3B-Instruct-Q6_K_L.gguf")
llm = Llama(model_path=downloaded_model_path, n_ctx=16384, n_gpu_layers=-1, verbose=False)

## Create Embeddings

In [2]:
prompt = f"""Generate five different Google search queries to find information on the top academic AI conferences in 2025.
            These queries should be tailored to discover high-impact conferences covering fields like machine learning, natural language processing,
            computer vision, and other areas within AI."""
queries = llm_generate_search_queries(llm, prompt)
urls = list({url['url'] for query in queries for url in get_top_urls_for_google_search_query(query)})
markdowns = scrape_and_convert_to_markdown(urls)

In [3]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
chunk_size = 500
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
documents = []
for markdown_document in markdowns:
    md_header_splits = markdown_splitter.split_text(markdown_document)
    documents .extend(text_splitter.split_documents(md_header_splits))

print(len(documents))  

1176


In [ ]:
batch_size = 100
documents_embeddings = []
for idx in range(0, len(documents), batch_size):
    document_batch = documents[idx:idx+chunk_size]
    embeddings = llm_embedding.create_embedding([item.page_content for item in document_batch])    
    documents_embeddings.extend([(document, embeddings['embedding']) for document, embeddings in zip(document_batch, embeddings['data'])])

## Store Embeddings into DB

In [11]:
client.create_collection(
    collection_name="conferences",
    vectors_config=VectorParams(size=llm_embedding.metadata['bert.embedding_length'], distance=Distance.COSINE),
)

True

In [12]:
points = [PointStruct(id = str(uuid.uuid4()), vector = embeddings, payload = {"text": doc.page_content}) for doc, embeddings in documents_embeddings]
operation_info = client.upsert(collection_name="conferences", wait=True, points=points)

## Query

In [ ]:
def llamacpp_with_rag(llm: Llama, llm_embedding: Llama, client: QdrantClient, prompt: str):
    query_vector = llm_embedding.create_embedding(prompt)['data'][0]['embedding']
    search_result = client.search(
      collection_name="conferences",
      query_vector=query_vector,
      limit=20
    )
    prompt = f"""You are an AI assistant. Based on the context below, answer the following question.
                Question: {prompt}\n
                {"\n\n".join([row.payload['text'] for row in search_result])}\n\n"""
    output = llamacpp_with_character_level_parser(llm, prompt, None)
    return output

In [5]:
prompt = "Please give me the top academic AI conferences in 2025."
output = llamacpp_with_rag(llm, llm_embedding, client, prompt)
output

'+ **Date:** To be announced\n+ **Location:** TBD\n+ **Website:** [WiML](https://www.wiml.org/)  \n+ **Date:** To be announced\n+ **Location:** TBD\n+ **Website:** [WiML](https://www.wiml.org/)  \n+ **Date:** To be announced\n+ **Location:** TBD\n+ **Website:** [WiML](https://www.wiml.org/)  \nConclusion\n----------  \nAttending these top AI conferences in 2025 will enhance your knowledge and expand your professional network. Make sure to check the official websites for the latest updates on dates and locations. Whether you are looking to dive deep into research or explore practical applications, there is a conference for everyone in the AI community.  \n### Related answers\n\n+ **Date:** To be announced\n+ **Location:** TBD\n+ **Website:** [WiML](https://www.wiml.org/)  \n+ **Date:** To be announced\n+ **Location:** TBD\n+ **Website:** [WiML](https://www.wiml.org/)  \n+ **Date:** To be announced\n+ **Location:** TBD\n+ **Website:** [WiML](https://www.wiml.org/)  \nConclusion\n--------

In [6]:
prompt = "Please give me the top acadamic AI conferences in 2025 sorted by date as json."
output = llamacpp_with_rag(llm, llm_embedding, client, prompt)
output

'------------------------------------](#cm23ndunu4j579srqlj3bino2)As we look ahead to 2025, the landscape of AI conferences is evolving rapidly, offering a plethora of opportunities for professionals and enthusiasts alike. Here’s a detailed overview of some of the most anticipated AI conferences to attend in 2025\\.  \nMajor AI Conferences\n--------------------  \n1. **NeurIPS 2025**\n\n\n\n## Step 1: Identify the top academic AI conferences\nTo provide the top academic AI conferences in 2025 sorted by date in JSON format, we first need to identify the conferences. Based on the provided sources, we can determine the key conferences.\n\n## Step 2: Determine the conference dates\nAfter identifying the conferences, we need to gather their respective dates. However, the provided information does not explicitly mention the dates of these conferences. For the sake of providing an answer, we\'ll assume the top academic AI conferences are NeurIPS, IJCAI, and ICML, based on general knowledge.\n

In [7]:
prompt = "Please give me the top academic AI conferences in 2025, organized in a sequential order that would allow me to attend each one? I'd like to visit as many as possible. "
output = llamacpp_with_rag(llm, llm_embedding, client, prompt)
output

'* [How to Choose the Right AI Conference in 2025](#cm23nduo94j599srqczwouwi9)\n* [Top AI Conferences to Attend in 2025](#cm29nsw2n62xquf2jeg7eqsel)\n* [Top AI Conferences to Attend in 2025](#cm2aa6ak2ctr1uf2j5l5agfy8)\n\nI can provide you with the information you requested. Here is a sequential list of top academic AI conferences in 2025, organized in a way that would allow you to attend each one:\n\n1.  **NeurIPS 2025**\n    *   Date: To be announced\n    *   Location: TBD\n    *   Website: [WiML](https://www.wiml.org/)\n2.  **NIPS 2025**\n    *   Date: To be announced\n    *   Location: TBD\n    *   Website: [WiML](https://www.wiml.org/)\n3.  **ICML 2025**\n    *   Date: To be announced\n    *   Location: TBD\n    *   Website: [WiML](https://www.wiml.org/)\n4.  **ICML 2025**\n    *   Date: To be announced\n    *   Location: TBD\n    *   Website: [WiML](https://www.wiml.org/)\n5.  **ICML 2025**\n    *   Date: To be announced\n    *   Location: TBD\n    *   Website: [WiML](https://www

In [8]:
client.close()
llm_embedding.close()